In [1]:
import pandas as pd

In [3]:
df = pd.read_pickle("../data/processed/train_data.pkl")

In [4]:
df.head(100)

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,hour,weekday,month,air_temperature_6_lag,air_temperature_24_lag,dew_temperature_6_lag,dew_temperature_24_lag
0,0,0,0.000000,0,0,8.913685,11.0,NaN,25.000000,6.0,20.000000,NaN,0,4,1,NaN,NaN,NaN,NaN
1,1104,1,0.000000,13,9,12.204687,NaN,NaN,-8.296875,8.0,-12.203125,NaN,0,4,1,13.132812,18.778646,9.699219,10.911784
2,1104,0,142.341003,13,9,12.204687,NaN,NaN,-8.296875,8.0,-12.203125,NaN,0,4,1,13.781250,18.986979,9.983073,11.074219
3,1103,0,43.320999,13,7,10.081801,NaN,NaN,-8.296875,8.0,-12.203125,NaN,0,4,1,14.796875,19.174479,9.983073,11.215820
4,1102,2,3234.379883,13,6,11.375822,NaN,NaN,-8.296875,8.0,-12.203125,NaN,0,4,1,16.179688,19.358073,10.183594,11.332682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1045,0,8.860000,12,6,10.420226,NaN,NaN,1.900391,4.0,-1.200195,NaN,0,4,1,16.399740,17.963542,11.936198,9.570638
96,1055,0,32.605000,12,0,11.370382,NaN,4.0,1.900391,4.0,-1.200195,NaN,0,4,1,17.884115,18.684570,11.666667,10.504883
97,1046,0,143.673004,12,0,12.212805,NaN,NaN,1.900391,4.0,-1.200195,NaN,0,4,1,15.850260,18.034505,12.119792,9.708333
98,1048,0,71.114998,12,0,12.526746,NaN,NaN,1.900391,4.0,-1.200195,NaN,0,4,1,14.816406,18.171875,12.036458,9.942057
